#Import Package<br>
備註: 此程式會將frame數分為兩個model去訓練，並分別向左右延伸，找到最佳的訓練與測試集

In [10]:
# -*- coding: utf-8 -*-
import os

import pandas as pd

from datetime import datetime
from datetime import timedelta
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from keras.utils import Sequence, plot_model
from keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")
#from models import *    #在jupyter中 改為 下面cell的 %run models.ipnyb
import tensorflow as tf
import random as rn
rn.seed(666)
tf.set_random_seed(666)


config = tf.ConfigProto()           # 此三行的目的是讓TensorFlow在運行過程中動態申請顯存，需要多少就申請多少
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

Import model<br>
包括 c3d、c3g25、c4g12、c5g6、c6g3、lrcn

In [11]:
%run models.ipynb

#Class Define

In [12]:
class DataGenerator(Sequence): 
    'Generates data for Keras'
    def __init__(self, list_IDs_1, list_IDs_3, labels, fnum, peak, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs_1 = list_IDs_1
        self.list_IDs_3 = list_IDs_3
        self.fnum = fnum
        self.shuffle = shuffle
        self.on_epoch_end()
        self.peak = peak

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs_1) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # Find list of IDs
        list_IDs_temp_1, y = [self.list_IDs_1[k] for k in indexes],\
                             [self.labels[k] for k in indexes]
        # Generate data
        X1 = self.__data_generation(list_IDs_temp_1, indexes)
        return X1, np.array(y, dtype=int)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs_1))
        if self.shuffle == True:
            np.random.seed(666)
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp_1, y_index):
        'Generates data containing batch_size samples'
        # Initialization
        X1 = []
        # Generate data
        for ID, y_ID in zip(list_IDs_temp_1, y_index):
            # Store sample
            x_1 = list(np.load(ID + '.npy'))
            if len(x_1) >= 120:
                x_1 = x_1[70:120]
            else:
                x_1 = x_1[:frame_num]
            while len(x_1) < self.fnum:
                x_1.append(np.zeros_like(x_1[0]))
            X1.append(x_1)
        X1 = np.array(X1)
        return X1
    
class FrameInterval:
    def __init__(self):
        self.train_df = pd.DataFrame(columns=['data'], dtype=str)
        self.test_df = pd.DataFrame(columns=['data'], dtype=str)
        self.val_df = pd.DataFrame(columns=['data'], dtype=str)

    def add_train(self, data, datatime):
        self.train_df.loc[datatime] = [data]
        self.train_df.index = pd.to_datetime(self.train_df.index)

    def add_test(self, data, datatime):
        self.test_df.loc[datatime] = [data]
        self.test_df.index = pd.to_datetime(self.test_df.index)

    def add_val(self, data, datatime):
        self.val_df.loc[datatime] = [data]
        self.val_df.index = pd.to_datetime(self.val_df.index)

#Define compare_mae

In [13]:
def compare_mae(new, train, test, frame_num, mae, model, stage, dir):
    global state
    print("Stage %d" % stage, file=open(total_name, 'a'))
    print("Current MAE:", "%.3f" % mae, file=open(total_name, 'a'))
    new_train = sorted(new + train)
    new_test = sorted(new + test)
    X_train = []
    y_train_index = []
    X_test = []
    y_test_index = []
    X_val = []
    y_val_index = []
    for t in new_test:
        X_test += FrameList[t].test_df['data'].tolist()
        y_test_index += FrameList[t].test_df.index
        X_val += FrameList[t].val_df['data'].tolist()
        y_val_index += FrameList[t].val_df.index
    for t in new_train:
        X_train += FrameList[t].train_df['data'].tolist()
        y_train_index += FrameList[t].train_df.index
    X_train_fnum = frame_df.loc[y_train_index]
    X_test_fnum = frame_df.loc[y_test_index]
    X_val_fnum = frame_df.loc[y_val_index]
    y_train = vd_df.loc[y_train_index, feature].values
    y_test = vd_df.loc[y_test_index, feature].values
    y_val = vd_df.loc[y_val_index, feature].values
    print("New Train Length:", len(X_train), file=open(total_name, 'a'))
    print("New Test Length:", len(X_test), file=open(total_name, 'a'))
    print("New Val Length:", len(X_val), file=open(total_name, 'a'))

    X_train = np.array(X_train)
    X_train_fnum = np.array(X_train_fnum)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    X_test_fnum = np.array(X_test_fnum)
    y_test = np.array(y_test)
    X_val = np.array(X_val)
    X_val_fnum = np.array(X_val_fnum)
    y_val = np.array(y_val)
    testing_generator = DataGenerator(X_test, X_test_fnum, y_test, frame_num, dir, False)

    mae_2, mae_3 = 100, 100
    if stage:
        testPrdct = model.predict_generator(generator=testing_generator, max_queue_size=500, workers=32, verbose=1)
        testPrdct = np.squeeze(testPrdct)
        mae_2 = abs(y_test[:len(testPrdct)] - testPrdct).mean()
        print("Merge Test MAE:", "%.3f" % mae_2, file=open(total_name, 'a'))

        plt.figure(num=None, figsize=(50, 5))
        plt.tick_params(labelsize=30)
        plt.plot(y_test, lw=2, label='Target')
        plt.plot(testPrdct, lw=2, label='Predict')
        plt.legend(loc=1, prop={'size': 20})
        plt.tight_layout()
        plt.grid()
        # plt.title(mae_2)
        plt.savefig(
            "plot/" + cctv_Id + "/" + dir + "/stage_%d_merge1.png" % stage)
        plt.close('all')
        plt.clf()
        plt.cla()
    if state == 0:
        training_generator = DataGenerator(X_train, X_train_fnum, y_train, frame_num, dir, True)
        validation_generator = DataGenerator(X_val, X_val_fnum, y_val, frame_num, dir, True)
        model_temp = model
        history = model_temp.fit_generator(generator=training_generator, validation_data=validation_generator,
                                           epochs=epochs, steps_per_epoch=len(X_train) // batch_size,
                                           validation_steps=len(X_val) // batch_size,
                                           max_queue_size=500, verbose=1, callbacks=[early_stopping], workers=32)
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(loc=1)
        plt.tight_layout()
        plt.grid()
        plt.savefig(
            "plot/" + cctv_Id + "/" + dir + "/loss.png")
        testPrdct_2 = model_temp.predict_generator(generator=testing_generator, max_queue_size=500, workers=32, verbose=1)
        testPrdct_2 = np.squeeze(testPrdct_2)
        mae_3 = abs(y_test[:len(testPrdct_2)] - testPrdct_2).mean()
        print("Merge All MAE:", "%.3f" % mae_3, file=open(total_name, 'a'))
        plt.figure(num=None, figsize=(50, 5))
        plt.tick_params(labelsize=30)
        plt.plot(y_test, lw=2, label='Target')
        plt.plot(testPrdct_2, lw=2, label='Predict')
        plt.tight_layout()
        plt.grid()
        # plt.title("MAE: %.3f" % mae_3)
        plt.legend(loc=1, prop={'size': 20})
        plt.savefig(
            "plot/" + cctv_Id + "/" + dir + "/stage_%d_merge2.png" % stage)
        plt.close('all')
        plt.clf()
        plt.cla()
    MAE = np.array([mae, mae_2, mae_3])
    mini = MAE.argmin()
    if stage == 0:
        state = 0
        print("State: Initial", file=open(total_name, 'a'))
        return new_train, new_test, mae_3, model_temp, False, testPrdct_2
    if mini == 0:
        state = 0
        print("State: Stop merging", file=open(total_name, 'a'))
        return train, test, mae, model, True, None
    elif mini == 1:
        state = 1
        print("State: Add test -> %d" % new[0], file=open(total_name, 'a'))
        return train, new_test, mae_2, model, False, testPrdct
    else:
        state = 0
        print("State: Add train & test -> %d" % new[0], file=open(total_name, 'a'))
        return new_train, new_test, mae_3, model_temp, False, testPrdct_2


#Basic setting<br>
day1, train_dayLast: 訓練時間<br>
test_day1, dayLast: 測試時間<br>
val_day1, val_dayLast: 驗證時間<br>
feature: 選擇真值(Spd_Max, Spd_Min, Spd_Avg)

In [14]:
vds = ['nfbVD-N5-N-16.900-M-PS', 'nfbVD-N5-N-18.313-M-PS', 'nfbVD-N5-N-22.510-M-PS', 'nfbVD-N5-N-25.310-M-PS',
       'nfbVD-N5-S-18.312-M-PS', 'nfbVD-N5-S-19.677-M-PS', 'nfbVD-N5-S-21.063-M-PS', 'nfbVD-N5-S-23.910-M-PS']

cctvs = ['nfbCCTV-N5-N-16.915-M', 'nfbCCTV-N5-N-18.308-M', 'nfbCCTV-N5-N-22.514-M', 'nfbCCTV-N5-N-25.309-M',
         'nfbCCTV-N5-S-18.339-M', 'nfbCCTV-N5-S-19.7-M', 'nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M']
vds = ["nfbVD-N5-S-21.063-M-PS", "nfbVD-N5-S-23.910-M-PS", "nfbVD-N5-N-25.310-M-PS"]
cctvs = ['nfbCCTV-N5-S-21.048-M', 'nfbCCTV-N5-S-23.896-M', 'nfbCCTV-N5-N-25.309-M']

features = ['Spd_Max', 'Spd_Avg', 'Spd_Min']

day1 = datetime.strptime('20190728 0600', "%Y%m%d %H%M")
day1_str = day1.strftime("%Y%m%d %H%M")
train_dayLast = datetime.strptime('20190729 2359', "%Y%m%d %H%M")

test_day1 = datetime.strptime('20190728 0600', "%Y%m%d %H%M")
test_day1_str = test_day1.strftime("%Y%m%d %H%M")
dayLast = datetime.strptime('20190729 2359', "%Y%m%d %H%M")
dayLast_str = dayLast.strftime("%Y%m%d %H%M")

val_day1 = datetime.strptime('20190729 0600', "%Y%m%d %H%M")
val_dayLast = val_day1 + timedelta(days=6) - timedelta(minutes=1)

# day1 = datetime.strptime('20190320 0600', "%Y%m%d %H%M")
# day1_str = day1.strftime("%Y%m%d %H%M")
# train_dayLast = datetime.strptime('20190320 0630', "%Y%m%d %H%M")
#
# test_day1 = datetime.strptime('20190328 0600', "%Y%m%d %H%M")
# test_day1_str = test_day1.strftime("%Y%m%d %H%M")
# dayLast = datetime.strptime('20190328 0630', "%Y%m%d %H%M")
# dayLast_str = dayLast.strftime("%Y%m%d %H%M")
# Add VDs
# dt_dtypes = {'Spd0': int, 'Spd1': int, 'Spd2': int, 'SpdMax': int, 'SpdMin': int, 'SpdAvg': int}
# train_dt_index = pd.date_range(day1, periods=(train_dayLast - day1).total_seconds() // 60 + 1, freq='T')
# test_dt_index = pd.date_range(test_dayFirst, periods=(dayLast - test_dayFirst).total_seconds() // 60 + 1, freq='T')

#Todo<br>
model: 要訓練的神經網路模型<br>
filename: 選擇vd真值的csv檔<br>
備註: 此程式會將frame數分為兩個model去訓練，並分別向左右延伸，找到最佳的訓練與測試集

In [20]:
# Todo: Tune the fitting parameters
feature = 'Spd_Max'
batch_size = 32
epochs = 100
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='auto',
                               verbose=0, restore_best_weights=True)
"""Choose model"""
name, model = c3g25()

total_name = 'plot/All_log.txt'
open(total_name, 'w').close()

for cctv_Id, vd_Id in zip(cctvs, vds):
    print(cctv_Id)
    print(name)
    print(cctv_Id, file=open(total_name, 'a'))
    vdDir = 'D:/vd/' + vd_Id + '/vd_speed/'
    cctvDir = 'D:/wcs/cctv/' + cctv_Id + "/"
    filename = vdDir + 'vdspd_20190728_20190729.csv'   #'vdspd_20190322_20190608.csv'
    total_frames = cctvDir + "frame_num.txt"
    vd_df = pd.read_csv(filename, infer_datetime_format=True, index_col=0)
    vd_df.index = pd.to_datetime(vd_df.index)
    vd_df = vd_df.shift(-1)
    vd_df.loc[vd_df.index[len(vd_df)-1]] = 90
    frame_df = pd.read_csv(total_frames, infer_datetime_format=True, index_col=0, header=None)
    frame_df.index = pd.to_datetime(frame_df.index)
    print(frame_df.index)
    total_test_index = []
    FrameList = []
    with open(cctvDir + "frame_interval.txt") as f:
        content = f.readlines()
    frame_interval = np.array([x.strip() for x in content], dtype=int)
    for i in range(len(content)):
        FrameList.append(FrameInterval())
    print(datetime.now())

    if not os.path.exists("plot/" + cctv_Id + "/peak_1"):
        os.makedirs("plot/" + cctv_Id + "/peak_1")
        os.makedirs("plot/" + cctv_Id + "/peak_2")
    else:
        for f in os.listdir("plot/" + cctv_Id + "/peak_1"):
            os.remove("plot/" + cctv_Id + "/peak_1/" + f)
        for f in os.listdir("plot/" + cctv_Id + "/peak_2"):
            os.remove("plot/" + cctv_Id + "/peak_2/" + f)
    train_count = 0
    test_count = 0
    val_count = 0
    day = day1
    while day <= train_dayLast:
        # print(day)
        if not os.path.exists(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + ".npy")\
                or day not in vd_df.index:
            day += timedelta(minutes=1)
            continue
        train_count += 1
        FrameList[int(frame_df.loc[day]) // 10]\
            .add_train(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M"), day)
        day += timedelta(minutes=1)

    day = test_day1
    while day <= dayLast:
        # print(day)
        if not os.path.exists(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + ".npy") \
                or day not in vd_df.index:
            day += timedelta(minutes=1)
            continue
        test_count += 1
        FrameList[int(frame_df.loc[day]) // 10] \
            .add_test(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M"), day)
        total_test_index.append(day)
        day += timedelta(minutes=1)

    day = val_day1
    while day <= val_dayLast:
        # print(day)
        if not os.path.exists(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M") + ".npy") \
                or day not in vd_df.index:
            day += timedelta(minutes=1)
            continue
        val_count += 1
        FrameList[int(frame_df.loc[day]) // 10] \
            .add_val(cctvDir + day.strftime("%Y%m%d") + "_" + day.strftime("%H%M"), day)
        day += timedelta(minutes=1)

    print("Total Train:", train_count, file=open(total_name, 'a'))
    print("Total test:", test_count, file=open(total_name, 'a'))
    print("Total val:", val_count, file=open(total_name, 'a'))
    print("Frame Num:", len(frame_df))
    shape = np.load(FrameList[5].train_df.iloc[0].values[0]+'.npy').shape[1:]
    #shape2 = np.load(FrameList[5].train_df.iloc[0].values[0]+'_timestamp.npy').shape[1:]
    # my_models = [build_model(shape), build_model_2(shape), build_model_3(shape)]

    ModelTrain_1 = [list(frame_interval).index(sorted(frame_interval)[-1])]
    ModelTrain_1 = [ModelTrain_1[0] - 1] + ModelTrain_1
    ModelTest_1 = ModelTrain_1.copy()

    if abs(list(frame_interval).index(sorted(frame_interval)[-2]) -
           list(frame_interval).index(sorted(frame_interval)[-1])) < 5:
        ModelTrain_2 = [list(frame_interval).index(sorted(frame_interval)[-3])]
    else:
        ModelTrain_2 = [list(frame_interval).index(sorted(frame_interval)[-2])]
    ModelTrain_2 = [ModelTrain_2[0] - 1] + ModelTrain_2
    ModelTest_2 = ModelTrain_2.copy()

    train = [ModelTrain_1, ModelTrain_2]
    test = [ModelTest_1, ModelTest_2]

    total_test = []

    for peaks in range(2):
        state = 0
        Index = train[peaks][0] - 1
        frame_num = train[peaks][-1]*10 if train[peaks][-1]*10 < 90 else 90
        frame_num = 50
        my_model = model
        print("\n*Peak %d" % (peaks+1), file=open(total_name, 'a'))
        train[peaks], test[peaks], currentMAE, my_model, isStopMerging, predict =\
            compare_mae([], train[peaks], test[peaks], frame_num, 100, my_model, 0, "peak_%d" % (peaks+1))
        # file_name = "plot/" + cctv_Id + "/" + test_day1_str + '_' + dayLast_str + '/log.txt'
        # open(file_name, 'w').close()
        stages = 0
        print("Frame Num:", frame_num, file=open(total_name, 'a'))
        print("Left Hand Side", file=open(total_name, 'a'))
        while Index >= 0 and not isStopMerging:
            stages += 1
            print(Index)
            print("Index:", Index, file=open(total_name, 'a'))
            if len(FrameList[Index].train_df) > 60 and len(FrameList[Index].val_df) > 0 \
                    and len(FrameList[Index].test_df) > 0:
                train[peaks], test[peaks], currentMAE, my_model, isStopMerging, p = \
                    compare_mae([Index], train[peaks], test[peaks], frame_num, currentMAE, my_model, stages, "peak_%d" % (peaks+1))
                if p is not None:
                    predict = p
            else:
                if len(FrameList[Index].test_df) > 0 and len(FrameList[Index].val_df) > 0:
                    state = 1
                    train[peaks], test[peaks], currentMAE, my_model, isStopMerging, p = \
                        compare_mae([Index], train[peaks], test[peaks], frame_num, currentMAE, my_model, stages,
                                    "peak_%d" % (peaks + 1))
                    if p is not None:
                        predict = p
                else:
                    print("No data!", file=open(total_name, 'a'))
                    print("Train List:", train[peaks], file=open(total_name, 'a'))
                    print("Test List:", test[peaks], file=open(total_name, 'a'))
                    print("", file=open(total_name, 'a'))
                    break
            print("Train List:", train[peaks], file=open(total_name, 'a'))
            print("Test List:", test[peaks], file=open(total_name, 'a'))
            print("", file=open(total_name, 'a'))
            Index -= 1
        state = 0
        isStopMerging = False
        print("Right Hand Side", file=open(total_name, 'a'))
        Index = train[peaks][-1] + 1
        while Index < 21 and not isStopMerging:
            stages += 1
            print(Index)
            print("Index:", Index, file=open(total_name, 'a'))
            if len(FrameList[Index].train_df) > 60 and len(FrameList[Index].val_df) > 0 \
                    and len(FrameList[Index].test_df) > 0:
                train[peaks], test[peaks], currentMAE, my_model, isStopMerging, p = \
                    compare_mae([Index], train[peaks], test[peaks], frame_num, currentMAE, my_model, stages, "peak_%d" % (peaks+1))
                if p is not None:
                    predict = p
            else:
                if len(FrameList[Index].test_df) > 0 and len(FrameList[Index].val_df) > 0:
                    state = 1
                    train[peaks], test[peaks], currentMAE, my_model, isStopMerging, p = \
                        compare_mae([Index], train[peaks], test[peaks], frame_num, currentMAE, my_model, stages,
                                    "peak_%d" % (peaks + 1))
                    if p is not None:
                        predict = p
                else:
                    print("No data!", file=open(total_name, 'a'))
                    print("Train List:", train[peaks], file=open(total_name, 'a'))
                    print("Test List:", test[peaks], file=open(total_name, 'a'))
                    print("", file=open(total_name, 'a'))
                    break
            print("Train List:", train[peaks], file=open(total_name, 'a'))
            print("Test List:", test[peaks], file=open(total_name, 'a'))
            print("", file=open(total_name, 'a'))
            Index += 1
        total_test += test[peaks]
        index = 0
        for t in test[peaks]:
            if len(FrameList[t].test_df) == 0:
                continue
            temp = predict[index:index+len(FrameList[t].test_df)].tolist()
            while len(temp) < len(FrameList[t].test_df):
                temp.append(temp[-1])
            index += len(FrameList[t].test_df)
            FrameList[t].test_df.insert(loc=0, column='predict', value=temp)
        print("------------------", file=open(total_name, 'a'))
        my_model.save("plot/" + cctv_Id + "/peak_%d" % (peaks+1) + "/model.h5")
    predict_df = pd.DataFrame(columns=['value'])
    for i in total_test_index:
        for j in total_test:
            if i in FrameList[j].test_df.index:
                predict_df.loc[i] = [FrameList[j].test_df.loc[i]['predict']]
    df = pd.DataFrame(index=total_test_index)
    predict_all = np.array(predict_df.loc[total_test_index, 'value'].values)
    vd_all = np.array(vd_df.loc[total_test_index, feature].values)
    df.insert(loc=0, column='target', value=vd_all)
    df.insert(loc=0, column='predict', value=predict_df)
    df = df.replace(r'\s+', np.nan, regex=True).fillna(method='ffill')
    df = df.replace(r'\s+', np.nan, regex=True).fillna(method='bfill')
    df.to_csv("plot/" + cctv_Id + "/total_result.csv")
    print("Total predict:", len(predict_df), file=open(total_name, 'a'))
    print("Total test:", len(vd_all), file=open(total_name, 'a'))
    y = np.array(df['target'].values)
    p = np.array(df['predict'].values)
    mae = abs(y - p).mean()
    std = abs(y - p).std()
    plt.figure(num=None, figsize=(50, 5))
    plt.title('MAE: %.2f, σ: %.2f' % (mae, std), fontsize=30)
    plt.tick_params(labelsize=30)
    plt.plot(y, label='Target')
    plt.plot(p, label='Predict')
    plt.legend(loc=3, prop={'size': 20})
    plt.tight_layout()
    plt.grid()
    # plt.title(mae_2)
    plt.savefig("plot/" + cctv_Id + "/total.png")
    plt.close('all')
    plt.clf()
    plt.cla()
    print("Total MAE: %.3f\n" % mae, file=open(total_name, 'a'))
    print("**********************\n", file=open(total_name, 'a'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 50, 72, 106, 1)    0         
_________________________________________________________________
conv3d_55 (Conv3D)           (None, 50, 36, 53, 32)    896       
_________________________________________________________________
batch_normalization_55 (Batc (None, 50, 36, 53, 32)    128       
_________________________________________________________________
max_pooling3d_37 (MaxPooling (None, 50, 18, 26, 32)    0         
_________________________________________________________________
activation_55 (Activation)   (None, 50, 18, 26, 32)    0         
_________________________________________________________________
conv3d_56 (Conv3D)           (None, 50, 18, 26, 32)    27680     
_________________________________________________________________
batch_normalization_56 (Batc (None, 50, 18, 26, 32)    128       
__________

3/3 [==============================] - 40s 13s/step - loss: 4.3524 - acc: 0.1562 - val_loss: 4.3626 - val_acc: 0.1354
Epoch 37/100
3/3 [==============================] - 40s 13s/step - loss: 4.4248 - acc: 0.1354 - val_loss: 4.2811 - val_acc: 0.1042
Epoch 38/100
3/3 [==============================] - 41s 14s/step - loss: 4.2860 - acc: 0.1250 - val_loss: 4.2708 - val_acc: 0.1042
Epoch 39/100
3/3 [==============================] - 41s 14s/step - loss: 4.2832 - acc: 0.1042 - val_loss: 4.2699 - val_acc: 0.1042
Epoch 40/100
3/3 [==============================] - 41s 14s/step - loss: 4.2776 - acc: 0.1042 - val_loss: 4.2667 - val_acc: 0.1042
Epoch 41/100
3/3 [==============================] - 40s 13s/step - loss: 4.2813 - acc: 0.1042 - val_loss: 4.2751 - val_acc: 0.1042
Epoch 42/100
3/3 [==============================] - 40s 13s/step - loss: 4.2821 - acc: 0.0833 - val_loss: 4.2729 - val_acc: 0.1042
Epoch 43/100
3/3 [==============================] - 40s 13s/step - loss: 4.2708 - acc: 0.1146 - 

8/8 [==============================] - 34s 4s/step
nfbCCTV-N5-S-23.896-M
C3G25
DatetimeIndex(['2019-07-29 16:41:00', '2019-07-29 16:42:00',
               '2019-07-29 16:43:00', '2019-07-29 16:44:00',
               '2019-07-29 16:45:00', '2019-07-29 16:46:00',
               '2019-07-29 16:47:00', '2019-07-29 16:48:00',
               '2019-07-29 16:49:00', '2019-07-29 16:50:00',
               ...
               '2019-07-30 16:32:00', '2019-07-30 16:33:00',
               '2019-07-30 16:34:00', '2019-07-30 16:35:00',
               '2019-07-30 16:36:00', '2019-07-30 16:37:00',
               '2019-07-30 16:38:00', '2019-07-30 16:39:00',
               '2019-07-30 16:40:00', '2019-07-30 16:41:00'],
              dtype='datetime64[ns]', name=0, length=1081, freq=None)
2019-08-07 05:51:17.766973
Frame Num: 1081
Epoch 1/100
3/3 [==============================] - 45s 15s/step - loss: 5.0309 - acc: 0.1146 - val_loss: 5.1599 - val_acc: 0.0833
Epoch 2/100
3/3 [==============================]

8/8 [==============================] - 98s 12s/step - loss: 2.8320 - acc: 0.1523 - val_loss: 2.5528 - val_acc: 0.1797
Epoch 38/100
8/8 [==============================] - 98s 12s/step - loss: 2.6039 - acc: 0.1914 - val_loss: 2.6475 - val_acc: 0.1250
Epoch 39/100
8/8 [==============================] - 98s 12s/step - loss: 3.0221 - acc: 0.1367 - val_loss: 2.3055 - val_acc: 0.1914
Epoch 40/100
8/8 [==============================] - 98s 12s/step - loss: 2.8277 - acc: 0.1406 - val_loss: 2.4320 - val_acc: 0.1758
Epoch 41/100
8/8 [==============================] - 98s 12s/step - loss: 2.5260 - acc: 0.1641 - val_loss: 5.0912 - val_acc: 0.0742
Epoch 42/100
8/8 [==============================] - 98s 12s/step - loss: 3.0011 - acc: 0.1445 - val_loss: 2.3267 - val_acc: 0.1797
Epoch 43/100
8/8 [==============================] - 99s 12s/step - loss: 2.6223 - acc: 0.1328 - val_loss: 2.8495 - val_acc: 0.1211
Epoch 44/100
8/8 [==============================] - 98s 12s/step - loss: 2.7457 - acc: 0.1250 - 

Epoch 43/100
2/2 [==============================] - 24s 12s/step - loss: 4.4403 - acc: 0.1094 - val_loss: 4.9259 - val_acc: 0.0469
Epoch 44/100
2/2 [==============================] - 25s 12s/step - loss: 4.7966 - acc: 0.1250 - val_loss: 4.5214 - val_acc: 0.0469
Epoch 45/100
2/2 [==============================] - 24s 12s/step - loss: 4.3649 - acc: 0.0625 - val_loss: 3.8980 - val_acc: 0.1250
Epoch 46/100
2/2 [==============================] - 24s 12s/step - loss: 4.3822 - acc: 0.1250 - val_loss: 3.9335 - val_acc: 0.0781
Epoch 47/100
2/2 [==============================] - 25s 12s/step - loss: 3.7770 - acc: 0.0938 - val_loss: 3.9275 - val_acc: 0.1094
Epoch 48/100
2/2 [==============================] - 24s 12s/step - loss: 4.0572 - acc: 0.0312 - val_loss: 3.9137 - val_acc: 0.1094
Epoch 49/100
2/2 [==============================] - 25s 12s/step - loss: 4.2532 - acc: 0.0156 - val_loss: 3.9242 - val_acc: 0.0938
Epoch 50/100
2/2 [==============================] - 24s 12s/step - loss: 3.9968 - a

8/8 [==============================] - 33s 4s/step
3
8/8 [==============================] - 33s 4s/step
6
8/8 [==============================] - 33s 4s/step
